# 현재 작업 중입니다.

<!-- # [Module 1.6] Horovod 훈련


본 워크샵의 모든 노트북은 `conda_python3` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 아래는 세이지메이커의 어떤 피쳐도 사용하지 않고, PyTorch 만을 사용해서 훈련 합니다. -->

# PyTorch CIFAR-10 local training  



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()

In [3]:
import os
import subprocess

instance_type = "local"

try:
    if subprocess.call("nvidia-smi") == 0:
        ## Set type to GPU if one is present
        instance_type = "local_gpu"
except:
    pass

print("Instance type = " + instance_type)

Instance type = local_gpu


### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [4]:
inputs = sagemaker_session.upload_data(path="../data", bucket=bucket, key_prefix="data/cifar10")
print("s3 inputs: ", inputs)

s3 inputs:  s3://sagemaker-ap-northeast-2-057716757052/data/cifar10


# Construct a script for training 
- epoch 10 , 20
    - 각각 테스트 정확도 55.2, 62.9


In [5]:
import os
import subprocess

instance_type = "local_gpu"
# instance_type = "ml.p3.8xlarge"

job_name ='cifar10-horovod'

## 시스템의 이전 도커 컨테이너 삭제
- 아래와 같은 명령어를 사용하여 저장 공간을 확보 합니다.

In [7]:
! df -h
! docker system prune -f --all
! df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        121G   76K  121G   1% /dev
tmpfs           121G  1.2M  121G   1% /dev/shm
/dev/xvda1      104G   97G  7.2G  94% /
/dev/xvdf       985G   43G  903G   5% /home/ec2-user/SageMaker
Deleted Containers:
d6b5072e9c99ac42f5ea809267c2649db615bdb953a31f9fda8bb556795a83a0

Deleted Networks:
sagemaker-local

Deleted Images:
untagged: 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference:1.6.0-gpu-py3
untagged: 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/pytorch-inference@sha256:25cb8433fd503ae5ba750a85173e084494816d7c0bf2390a742e8d15fda3af5c
deleted: sha256:e3593f043c58a220c4a67268e9a95d987f36e8bd234880704168fa916d0ae754
deleted: sha256:7d769182f6fcc23c371f49b733f3904ee5eeac8a3789f0f1ae63de4688421c02
deleted: sha256:4e6e1f0287ce0e3cfbe0c3aebf6ca610b5abc846ed7abfecfb2176d68b2a7a90
deleted: sha256:4df6cc60e83cc4ba0295155373cbe7e7038426d63c9289ad33b00a9f7cddcc97
deleted: sha256:9b777ad45036c17d497844c26c9007cab188f39

In [8]:
from sagemaker.pytorch import PyTorch

cifar10_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='source',    
    base_job_name = job_name,
    role=role,
    framework_version='1.6.0',
    py_version='py3',
    train_instance_count=1,
    train_instance_type=instance_type,
    hyperparameters={"epochs": 3, 
                     'lr': 0.001,
                     'batch-size': 64,
                     "backend": "gloo",                     
                    },    
)
cifar10_estimator.fit({"training" : inputs})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Creating network "sagemaker-local" with the default driver
Creating 9bseq7xuf5-algo-1-y97ft ... 
Creating 9bseq7xuf5-algo-1-y97ft ... done
Attaching to 9bseq7xuf5-algo-1-y97ft
9bseq7xuf5-algo-1-y97ft | 2021-07-29 06:08:41,052 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
9bseq7xuf5-algo-1-y97ft | 2021-07-29 06:08:41,095 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
9bseq7xuf5-algo-1-y97ft | 2021-07-29 06:08:41,098 sagemaker_pytorch_container.training INFO     Invoking user training script.
9bseq7xuf5-algo-1-y97ft | 2021-07-29 06:08:41,225 sagemaker-training-toolkit ERROR    Reporting training FAILURE
9bseq7xuf5-algo-1-y97ft | 2021-07-29 06:08:41,225 sagemaker-training-toolkit ERROR    framework error: 
9bseq7xuf5-algo-1-y97ft | Traceback (most recent call last):
9bseq7xuf5-algo-1-y97ft |   File "/opt/conda/lib/python3.6/site-packages/sagemaker_training/trainer.py", line 85, in train
9bseq7xuf5-algo-

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpsn9cx4zf/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

In [ ]:
from sagemaker.pytorch import PyTorch

instance_type

cifar10_estimator = PyTorch(
    entry_point="train_horovod.py",    
    source_dir='source',    
    base_job_name = job_name,
    role=role,
    framework_version='1.6.0',
    py_version='py3',
    train_instance_count=1,
    train_instance_type=instance_type,
    hyperparameters={"epochs": 3, 
                     'lr': 0.001,
                     'batch-size': 64,
                     "backend": "gloo",                     
                    },    
)
cifar10_estimator.fit({"training" : inputs})

In [ ]:
horovod_artifact_path = cifar10_estimator.model_data
print("horovod_artifact_path: ", horovod_artifact_path)


%store horovod_artifact_path

In [ ]:
! aws s3 ls {horovod_artifact_path} --recursive